In [3]:
import json
import random
import uuid
import re
import os
import json
import pandas as pd
from secgym.utils import LLM_call

SEPARATOR = "❖"

def generate_ip():
    return ".".join(str(random.randint(0, 255)) for _ in range(4))

def extract_ip_from_string(input) -> list:
    return re.findall(r'[0-9]+(?:\.[0-9]+){3}', input)

def extract_ipv6_from_string(input_string):
    # Updated regular expression for matching valid IPv6 addresses with at least 5 colons and handling "::"
    ipv6_pattern = r'\b(?:[0-9A-Fa-f]{1,4}:){2,7}[0-9A-Fa-f]{1,4}::?(?:[0-9A-Fa-f]{1,4})?\b'
    
    # Find all matches in the input string
    return re.findall(ipv6_pattern, input_string)

def generate_ipv6(original_ipv6):
    # random select 3 char and replace with random allow char
    new_ipv6 = original_ipv6

    for i in range(5):
        index = random.randint(0, len(new_ipv6)-1)
        if new_ipv6[index].isalnum():
            new_ipv6 = new_ipv6[:index] + random.choice('0123456789ABCDEF') + new_ipv6[index+1:]
    
    return new_ipv6

# ee17abf2-35a2-4a16-9850-89ebb4f499d0  generate this type
def generate_uuid():
    return str(uuid.uuid4())

def extract_uuid_from_string(input) -> list:
    if not isinstance(input, str):
        print("Convert to string:", input)
        input = str(input)
    return re.findall(r'[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}', input)

def extract_mac_address_from_string(input) -> list:
    return re.findall(r'(?:[0-9A-Fa-f]{2}[:-]){5}[0-9A-Fa-f]{2}', input)

def generate_mac():
    return ':'.join([random.choice('0123456789ABCDEF') + random.choice('0123456789ABCDEF') for i in range(6)]) # correct

def extract_sid_from_string(input) -> list:
    return re.findall(r'S-[0-9-]+', input)

def generate_sid(original_sid):
    # random select 3 number and replace with random number
    new_sid = original_sid

    for i in range(10):
        index = random.randint(0, len(new_sid)-1)
        if new_sid[index].isdigit():
            new_sid = new_sid[:index] + str(random.randint(0, 9)) + new_sid[index+1:]

    return new_sid

def extract_sharepoint_url_account_from_string(input) -> list:
    matches = re.findall(r'sharepoint.com/personal/([^/]+)_vnevado_', input)
    if len(matches) == 0:
        matches = re.findall(r'sharepoint.com/personal/([^/]+)/', input)
    return matches

def match_latitute_longitude(input:str):
    lat =  re.findall(r'"latitude":([-+]?\d*\.?\d+),', input)
    lon =  re.findall(r'"longitude":([-+]?\d*\.?\d+)}', input)
    return lat+lon

def anony_lat_lon(input:str):
    # random perturbation
    # convert to float
    ninput = float(input)
    if ninput == 0:
        return str(ninput)
    
    ninput += random.uniform(-4, 4)
    return str(ninput)

def extract_email_from_string(input) -> list:
    # if single 
    return re.findall(r'[\w\.-]+@[\w\.-]+', input)


import re

def check_u_starts_and_ends_with_number(s):
    # Use regular expression to check the pattern
    # ^u: starts with 'u'
    # \d+$: ends with one or more digits
    if re.match(r'^u\d+$', s):
        return True
    return False

In [4]:
with open("./hashlist.json") as f:    
    hashlist = json.load(f)



for d in hashlist:
    if d.isdigit():
        print(d)


61285056513
1


In [6]:
with open("./hashlist.json", "r") as f:
    data = json.load(f)
    
classified_list = {
        "ip": {},
        "ipv6": {},
        "uuid": {},
        "other": {},
        "uid": {},
        "sid": {},
        "mac": {},
        "hash36": {},
        "hash40": {},
        "longhash": {},
        "exempt": {}

}

exempt_list = ["hidden", '.xlsm', '.pdf', "anonymous logon","VNEVADO-", "sunny_", "dwm-", "vnevado-", "bright_", "txt"]

for k, v in data.items():
    if k == v:
        continue
    
    # match uuid
    uuids = extract_uuid_from_string(k)
    if len(uuids) > 0:
        classified_list["uuid"][k] = v
        continue

    # match ip
    ips = extract_ip_from_string(k)
    if len(ips) > 0:
        classified_list["ip"][k] = v
        continue

    # match ipv6
    ipv6s = extract_ipv6_from_string(k)
    if len(ipv6s) > 0:
        classified_list["ipv6"][k] = v
        continue

    # match uid
    if check_u_starts_and_ends_with_number(k):
        classified_list["uid"][k] = v
        continue

    # match sid
    sids = extract_sid_from_string(k)
    if len(sids) > 0:
        classified_list["sid"][k] = v
        continue

    # match mac
    macs = extract_mac_address_from_string(k)
    if len(macs) > 0:
        classified_list["mac"][k] = v
        continue

    if len(k) == 36 or len(k) == 37:
        classified_list["hash36"][k] = v
        continue
    
    if len(k) == 40 and "_" in k:
        classified_list["hash40"][k] = v
        continue

    if "RgAAAA" in k or "LgAAAA" in k:
        classified_list["longhash"][k] = v
        continue

    if any([exempt in k for exempt in exempt_list]) or k.isdigit():
        classified_list["exempt"][k] = v
        continue
    
    classified_list["other"][k] = v

In [7]:
print(len(classified_list['other']))

261


In [8]:
classified_list['other']

{'msftadmin': 'alexjones',
 'debrab': 'fk8mq',
 'jonaw': 'tgs2z',
 'lynner': 'dp5hn',
 'Mario Rogers': 'Derek Lee',
 'Violet Martinez': 'Lila Johnson',
 'violetm': 'kelseyq',
 'meganb': 'alyssat',
 'Megan Bower': 'Hailey Johnson',
 'jonis': 'taylorz',
 'MSFT Adm': 'MSFT Rae',
 'Jonathan Wolcott': 'Ethan Brooks',
 'adtoaadsyncserviceaccount': 'plkamdsyncserviceaccount',
 'pradeepg': 'k1pt7ez',
 'Pradeep Gupta': 'Ryan Luis',
 'marior': '4qs6v9m',
 'Joni Sherman': 'Ava Cooper',
 'adsyncmsa5513f$': 'adsyncmsa7298j$',
 'pgustavo': 'pwilson',
 'adfsadmin': 'gsmith',
 'pattif': 'bwalker',
 'kdickens': 'bjenkins',
 'lidiah': 'santiago',
 'ADSyncMSA43414$@internal.niseko.alpineskihouse.co': 'oprtMSA23122$@internal.niseko.alpineskihouse.co',
 'gADFS$@INTERNAL.NISEKO.ALPINESKIHOUSE.CO': 'pLOJK$@INTERNAL.NISEKO.ALPINESKIHOUSE.CO',
 'contosoadmin': 'uiopcmd',
 'mb-adfs$': 'hj-klmn$',
 'gadfs$': 'nklmn$',
 'ADSyncMSA5513f$@vnevado.alpineskihouse.co': 'LpSyncXYZ5315r$@vnevado.alpineskihouse.co',
 'ad

In [6]:
# classified_list["exempt"]
# classified_list["longhash"]
# classified_list["uid"]

In [9]:
len(classified_list["ip"])

156838

In [10]:
# merge other, uuid, ip, ipv6, sid, mac

filtered_list = {}
filtered_list.update(classified_list["other"])
filtered_list.update(classified_list["uuid"])
filtered_list.update(classified_list["ip"])
filtered_list.update(classified_list["ipv6"])
filtered_list.update(classified_list["sid"])
filtered_list.update(classified_list["mac"])

del filtered_list['man']
del filtered_list['1HqnlWTo2sJGugNw19js9K7dRPb6Wjhh42qt8VuXNJ81']
del filtered_list['LOGIN']
del filtered_list['00000000000000E0']
del filtered_list['NOT-FOUND']
del filtered_list['not-found']
del filtered_list['AdminWin1']
del filtered_list['User1']
del filtered_list['User2']
del filtered_list['User3']
del filtered_list['root']
del filtered_list['x123']

In [11]:
len(filtered_list)

164888

In [9]:
with open("./pii/filtered_hashlist.json", "w") as f:
    json.dump(filtered_list, f, indent=4)

In [11]:
import re

def check_u_starts_and_ends_with_number(s):
    # Use regular expression to check the pattern
    # ^u: starts with 'u'
    # \d+$: ends with one or more digits
    if re.match(r'^u\d+$', s):
        return True
    return False

# Example usage
test_strings = ['u648', 'u259', 'uTest', 'u123a', 'Test123', 'u100']
results = {s: check_u_starts_and_ends_with_number(s) for s in test_strings}
print(results)

{'u648': True, 'u259': True, 'uTest': False, 'u123a': False, 'Test123': False, 'u100': True}


In [29]:
len("f21534f9912d43dc9bc36036b82a47cdbright_9")

40

In [26]:
len("b3e84f7852194ab5b541726d7f6eb04au849")

36

In [19]:
a = {
    'a61ec61284c441028a055860828300a4u926': 'd39fk12349gk413jff4gjei9938dhks9u758',
    '3936a6c434ee4ebbbdf5efea66b82221u923': '2938lfa939ee3nejddk3jfna49fjdej9u387',
    '6b1f2c31438942ae84e2b3436a4fd12au901': '5k3l3d2893743jfdk3k2j3fks9dk3jasu543',
    '21ba310136d1404eb3a6368ec9e38be2u911': 'fa848dk439fj3kj438jf394fj43k3j5bu630',
    '99e6497ed6ea4d08a10fd144574f6194u931': '838djf84845kd8fkd3kfj93mfj942k3au857',
    'f4931c585d5f47bb8aa0cd374b7f087bu945': '9dkfj3948djf9429jf3j9f3kf9sok3jdu294',
    'd6a8c1f7fe0a4433ba96a5228d872cdbu929': '3f8de2j6hbc345dfgh2b8723cb9cud932',
    'b44a555a64104f87bf563d6bbc14074cu922': 'c874534a91je7df98ab564f6fe451c3fu734',
    '7acf2300a6394cef89baba0027c7a82cu946': '1d7f93e21cd395gha29dc9c701d7b92eu215',
    'f0fdd91ad11a4dc78f12c00bbad86030u914': '9bcee65ab8317gfe9283a22jd746580du147',
    'a28bee687478481caa10b32d4efbefa7u941': 'e836zr102df9423cga84b43e6ba8df3au218',
    '671d464882ae4bc4befe76f133a38ebcu939': '671d464882ae4bc4befe76f133a38ebcu405',
    '22f8a4e041fd437aa2d2e004309dd697u917': '22f8a4e041fd437aa2d2e004309dd697u809',
    '8712dd8ffd604c3f953027fdd1c36688u937': '8712dd8ffd604c3f953027fdd1c36688u901',
    '7e1986aada044bd5a5837f2532fe2a82u909': '7e1986aada044bd5a5837f2532fe2a82u302',
    '4061e66764244c208e8ff21689af66b7u1056': '2b3e9f245b1e4d348a7aa65467fe33a1u2045',
}

for k in a:
    print(len(k))

36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
37
